## Lab 4: DDI using ML

### Imports

In [16]:
import os
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.parse.corenlp import CoreNLPDependencyParser
from nltk.corpus import stopwords 
import xml.etree.ElementTree as ET

### Variables

In [20]:
devel_oath = '../../data/Devel'
test_path = '../../data/Test-DDI'
train_path = '../../data/Train'
outputfile = 'task9.2_develGoal_1.txt'

my_parser = CoreNLPDependencyParser(url="http://localhost:9000")

### Functions

In [26]:
def analyze(sent):
    if len(sent)<= 0:
        return None
    mytree, = my_parser.raw_parse(sent)
    tree = mytree.nodes
    ini_token = 0
                   
    # clean tree
    aux = ['address', 'head', 'lemma', 'rel', 'word', 'tag']
    for k in range(1,len(tree)):
        node = tree[k] 
        for key in list(node):
            if key not in aux:
                del node[key]
        
        # add offset
        ini_token = sent.find(node['word'] ,ini_token)
        
        node['start'] = ini_token
        ini_token += len(node['word'])
        node['end'] = ini_token - 1
    return tree

In [27]:
def extract_features(tree, entities, e1,e2):
    features = []
    
    for key in tree.keys():
        node = tree[key]
        print(node)
        
        feature_vector = [
            # Features
        ]
        features.append(feature_vector)
      
    return features

In [35]:
def output_features(sent_id, e1, e2, type_rel,features, outf):
    j = 0
    if():
                
    else:
        outf.write(sent_id+'\t'+e1+'\t'+e2+'\t'+gold_class+'\t'+features_str)
        outf.write("\n")

IndentationError: expected an indented block (<ipython-input-35-9f14dbe57852>, line 5)

In [33]:
def main_extract_features(inputdir, outputfile):
    outf = open(outputfile, "w")
    # process each file in directory
    for filename in os.listdir(inputdir):  
        # parse XML file, obtaining a DOM tree
        fullname = os.path.join(inputdir, filename)
        tree = ET.parse(fullname)
        root = tree.getroot()    
        
        # process each sentence in the file
        for sentence in root.findall('sentence'):
            
            sent_id = sentence.get('id')
            
            # load sentence entities into a dictionary
            entities = {}
            for ent in sentence.findall('entity'):
                ent_id = ent.get('id') 
                offs = ent.get('charOffset').split('-')
                entities[ent_id] = offs
            
            tree = analyze(sentence.get('text'))
            # for each pair in the sentence, decide whether it is DDI and its type
            for pair in sentence.findall('pair'):
                e1 = pair.get('e1')
                e2 = pair.get('e2')
            
                features = extract_features(tree, entities, e1,e2)
                output_features(sent_id, e1, e2, type_rel, features)
            
    outf.close()
    return

In [34]:
main_extract_features(train_path, 'train_feature_vectors')
main_extract_features(devel_path, 'devel_feature_vectors')
main_extract_features(test_path, 'test_feature_vectors')

{'address': 0, 'word': None, 'lemma': None, 'ctag': 'TOP', 'tag': 'TOP', 'feats': None, 'head': None, 'deps': defaultdict(<class 'list'>, {'ROOT': [4]}), 'rel': None}
{'address': 1, 'word': 'In', 'lemma': 'in', 'tag': 'IN', 'head': 4, 'rel': 'mark', 'start': 0, 'end': 1}
{'address': 4, 'word': 'provide', 'lemma': 'provide', 'tag': 'VB', 'head': 0, 'rel': 'ROOT', 'start': 12, 'end': 18}
{'address': 2, 'word': 'order', 'lemma': 'order', 'tag': 'NN', 'head': 1, 'rel': 'mwe', 'start': 3, 'end': 7}
{'address': 3, 'word': 'to', 'lemma': 'to', 'tag': 'TO', 'head': 4, 'rel': 'mark', 'start': 9, 'end': 10}
{'address': 5, 'word': 'information', 'lemma': 'information', 'tag': 'NN', 'head': 4, 'rel': 'dobj', 'start': 20, 'end': 30}
{'address': 6, 'word': 'for', 'lemma': 'for', 'tag': 'IN', 'head': 11, 'rel': 'case', 'start': 32, 'end': 34}
{'address': 11, 'word': 'labeling', 'lemma': 'labeling', 'tag': 'NN', 'head': 5, 'rel': 'nmod', 'start': 67, 'end': 74}
{'address': 7, 'word': 'the', 'lemma': '

NameError: name 'type_rel' is not defined

## MaxEntropy Classifier
### Learner

In [17]:
def read_features_and_classes(inputfile):
    features = []
    classes = []
    prev_sent_id = ''
    with open(inputfile) as f:
        for i, line in enumerate(f):
            saved_features = line.split()
            sent_id = saved_features[0]
            
            if i == 0:
                feature_vector = []
                classes_vector = []
                feature_vector.append(saved_features[5:])
                classes_vector.append(saved_features[4])
            
            elif sent_id == prev_sent_id:
                feature_vector.append(saved_features[5:])
                classes_vector.append(saved_features[4])
            
            else:
                features.append(feature_vector)
                classes.append(classes_vector)
                feature_vector = []
                classes_vector = []
            
            prev_sent_id = sent_id
    
    return features, classes               

In [18]:
def train(features_file, model_name):
    # Get features of train data
    features_train, gs_train = read_features_and_classes(features_file)
    numIterations = 100
 
    algorithm = nltk.classify.MaxentClassifier.ALGORITHMS[0]
    classifier = nltk.MaxentClassifier.train(features_train, algorithm, max_iter=numIterations)
    classifier.show_most_informative_features(10)

    
    return

In [ ]:
train('train_feature_vectors', 'ml_model.megam')

### Classifier